<a href="https://colab.research.google.com/github/lamld203844/kapur-and-otsu-segmentation/blob/main/gradio_GUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q gradio

In [2]:
# Install watermark
!pip install -q watermark

In [3]:
# Load watermark
%load_ext watermark

# Display the version of Python and Gradio
%watermark -v -p gradio,cv2,numpy,matplotlib,PIL,scipy,skimage

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

gradio    : 4.15.0
cv2       : 4.8.0
numpy     : 1.23.5
matplotlib: 3.7.1
PIL       : 9.4.0
scipy     : 1.11.4
skimage   : 0.19.3



In [4]:
import gradio as gr
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

from scipy.ndimage import median_filter
from skimage.filters import threshold_multiotsu
from skimage.filters import threshold_otsu

%matplotlib inline

In [5]:
def calculate_psnr(input_image):
    # Convert the input image to grayscale
    gray_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)

    # Calculate MSE
    mse = np.mean((gray_image.astype(np.float64) / 255) ** 2)
    if mse == 0:
        return "Same Image"

    # Calculate PSNR
    PIXEL_MAX = 255.0
    psnr = 20 * np.log10(PIXEL_MAX) - 10 * np.log10(mse)

    return psnr


def plot_histogram(img):
    # Flatten the image into 1 dimension: change shape (a,b,c) to (abc,)
    pixels = img.flatten()

    # Create histogram
    fig, ax = plt.subplots()
    ax.hist(pixels, bins=256, range=[0,256], color ='black', alpha=0.7)

    ax.set_title('Histogram')
    ax.set_xlabel('Pixel Intensity')
    ax.set_ylabel('Frequency')

    plt.close(fig)  # Close the figure so it won't display in your Python environment

    return fig


def basic_preprocessing(input_image, kernel_size=3):
    """
    Preprocessing: average, median filter, and histogram equalization with size and
    shape of the mask should be selected.((kernel_size = 3 by default))
    """

    # Convert the input image to grayscale
    gray_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)

    # Apply average filter
    blurred_image = cv2.blur(gray_image, (kernel_size, kernel_size))

    # Apply median filter
    median_filtered_image = median_filter(blurred_image, size=kernel_size)

    return blurred_image, median_filtered_image

In [6]:
def kapur_segment(input_image):
    # Convert the input image to grayscale
    gray_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)

    # Normalize to 0 - 255
    image = cv2.normalize(gray_image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

    # Apply Kapur's method
    thresholds = threshold_multiotsu(image)

    # Apply the thresholds to get a segmented image
    segmented_image = np.digitize(image, bins=thresholds)

    # Map the segmented image from 0-255
    segmented_image = ((segmented_image / segmented_image.max()) * 255).astype(np.uint8)

    # Convert the segmented image to a 3-channel image
    segmented_3_channel = cv2.cvtColor(segmented_image, cv2.COLOR_GRAY2BGR)

    return segmented_3_channel

In [7]:
def otsu_segment(input_image):
    # Convert the input image to grayscale
    gray_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)

    # Normalize to 0 - 255
    image = cv2.normalize(gray_image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

    # Apply Otsu's thresholding
    thresh = threshold_otsu(image)
    binary = image > thresh

    return binary

In [8]:
def main_function(img):

    # basic img information
    info = f"Type of image {type(img)}, shape = {img.shape}"

    # Display histogram
    fig = plot_histogram(img)

    # basic preprocessing
    average_filtered_img, median_filtered_img = basic_preprocessing(img)

    # psnr value
    psnr = calculate_psnr(img)
    psnr_value = f'PSNR value = {psnr}'

    # segmentation Kapur method
    kapur_seg_mask = kapur_segment(img)

    # segmentation otsu method
    otsu_seg_mask = otsu_segment(img)
    otsu_img = cv2.cvtColor((otsu_seg_mask * 255).astype(np.uint8), cv2.COLOR_GRAY2BGR) # Convert binary mask to a 3-channel image


    return info, fig, average_filtered_img, median_filtered_img, psnr_value, kapur_seg_mask, otsu_img

In [ ]:
inputs = [
    gr.Image(type='numpy', label="Input original image"),
    # gr.Image(type='numpy', label="Input original mask"),
]
outputs = [gr.Textbox(label="Basic image information"), # img info
            gr.Plot(label="img histogram"), # histogram
            gr.Image(label="avg filtering"), # avg filtering
            gr.Image(label="median filtering"), # median filtering
            gr.Textbox(label="psnr value"), # psnr value
            gr.Image(label="kapur_segmentation"), # segmentation Kapur method
            gr.Image(label="otsu_segmentation"), # segmentation Otsu method
        ]

iface = gr.Interface(fn=main_function,
                     inputs=inputs,
                     outputs=outputs)

iface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://ffab91264fcdf09c64.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
iface.close()